In [3]:
import pandas as pd
import numpy as np
import random
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import OneHotEncoder

from sklearn.metrics import mean_squared_error, mean_absolute_error,r2_score
from sklearn.linear_model import LinearRegression
import time
import datetime

In [4]:
tripdata = pd.read_csv("../../dataset/final_bike_sharing.csv")

In [5]:
tripdata.head(3)

,index,tripduration,start_station_id,start_station_name,start_lat,start_lon,end_station_id,end_station_name,end_lat,end_lon,...,conditions,date,dist,birthyear,years_old,holiday,day,month,seasons,gender
0,259617,1.02,others,Manila & 1st,40.721651,-74.042884,others,Manila & 1st,40.721651,-74.042884,...,Clear,2018-09-05,1.783012e-12,1994,24,working_day,weekday,September,autumn,male
1,283363,1.02,others,Heights Elevator,40.748716,-74.040443,others,Heights Elevator,40.748716,-74.040443,...,cloudy_rain,2018-10-06,1.782506e-12,1994,24,working_day,weekend,October,autumn,male
2,169168,1.02,others,Van Vorst Park,40.718489,-74.047727,es_3185,City Hall,40.717732,-74.043845,...,Clear,2018-07-09,3.384331e-01,1963,55,working_day,weekday,July,summer,female


In [6]:
tripdata.columns

Index(['index', 'tripduration', 'start_station_id', 'start_station_name',
       'start_lat', 'start_lon', 'end_station_id', 'end_station_name',
       'end_lat', 'end_lon', 'bikeid', 'usertype', 'hour', 'min', 'temp',
       'feelslike', 'dew', 'humidity', 'precip', 'precipprob', 'snow',
       'snowdepth', 'windspeed', 'winddir', 'sealevelpressure', 'cloudcover',
       'visibility', 'solarradiation', 'uvindex', 'conditions', 'date', 'dist',
       'birthyear', 'years_old', 'holiday', 'day', 'month', 'seasons',
       'gender'],
      dtype='object')

## Selecting variables according to P values

In [7]:
selected_features = ['tripduration','start_lat','start_lon','end_lat','end_lon','usertype','hour',
                     'temp','feelslike','dew','snowdepth','winddir','sealevelpressure','visibility','solarradiation',
                     'conditions','dist','birthyear','holiday','day','month','gender']

In [8]:
df = tripdata[selected_features]

In [9]:
def split_dataset(df):
    X = df.drop('tripduration', axis=1)
    y = df['tripduration']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    print('Train', X_train.shape, y_train.shape)
    print('Test', X_test.shape, y_test.shape)
    return X_train, X_test, y_train, y_test

### AMputation with 10% NA values

In [10]:
## get the sample size such as 5%, 10%,20% etc
def getSampleSize(df, perc=1):
    return round(perc/100 * df.shape[0])

def getAnIndex(index):
    li = []
    for i in index:
        li.append(i)
    return li   

def induceMissingValues(X_train,col_name, perc):
    random.seed(100)
    index = getAnIndex(X_train.index)
    sample_size = getSampleSize(X_train, perc)
    print("sample size is =", sample_size)
    selected_index = random.sample(index, sample_size)
    for i in selected_index:
        X_train.loc[i, col_name]= np.NaN
    return X_train

###====================####
def convert_to_dataframe(y_train):
    return pd.DataFrame(y_train)

def convert_to_series(y_train):
    return y_train.squeeze()

def induceMissingValuesForDrop(X_train,y_train,col_name, perc):
    random.seed(100)
    index = getAnIndex(X_train.index)
    sample_size = getSampleSize(X_train, perc)
    print("sample size is =", sample_size)
    selected_index = random.sample(index, sample_size)
    y_train = convert_to_dataframe(y_train)
    for i in selected_index:
        y_train.loc[i] = np.NaN
        X_train.loc[i,col_name]= np.NaN
    y_train = convert_to_series(y_train)
    return X_train, y_train

In [11]:
def sample_imputation_X(var,X_train,X_test):
    # extract a random sample
    random_sample_train = X_train[var].dropna().sample(X_train[var].isnull().sum(), random_state=0)
    random_sample_test = X_test[var].dropna().sample(X_test[var].isnull().sum(), random_state=0)
     # re-index the randomly extracted sample
    random_sample_train.index = X_train[X_train[var].isnull()].index
    random_sample_test.index = X_test[X_test[var].isnull()].index

    # replace the NA
    X_train.loc[X_train[var].isnull(), var] = random_sample_train
    X_test.loc[X_test[var].isnull(), var] = random_sample_test
    return X_train, X_test

In [12]:
fill_dict ={'start_lat': 40.79865054, 'start_lon': -74.2404433, 'end_lat': 40.79165072 , 'end_lon':-74.28593088,'usertype':'agent',
            'hour': 24, 'temp': 38.6, 'feelslike':49.4,'dew':-25.7,'snowdepth':5.10,'winddir':380.0,'sealevelpressure':819.3,
            'visibility':7.2,'solarradiation': 1109,'conditions': 'muddy','dist': 5.345,'birthyear': 2003,
            'holiday':'unknown','day': 'special','month': 'unknown','gender': 'nonbinary'
        }

In [13]:
def ampute_each_variables(variable, method, X_train, y_train, X_test, y_test):
    if method == "remove":
        X_train , y_train = induceMissingValuesForDrop(X_train,y_train,variable,perc=10)
        print('Train set after Amputation', X_train.shape, y_train.shape)
        print("y_train Amputed value count of varaible ", y_train.isnull().sum())
        print("X_train Amputed value count of varaible ", X_train.isnull().sum()[variable])
        X_test , y_test = induceMissingValuesForDrop(X_test,y_test,variable,perc=10)
        print('Test set after Amputation', X_test.shape, y_test.shape)
        print("y_test Amputed value count of varaible ", y_test.isnull().sum())
        print("X_test Amputed value count of varaible ", X_test.isnull().sum()[variable])
    else:
        X_train  = induceMissingValues(X_train,variable,perc=10)
        print('Train set after Amputation', X_train.shape, y_train.shape)
        print("X_train Amputed value count of varaible ", X_train.isnull().sum()[variable])
        X_test  = induceMissingValues(X_test,variable,perc=10)
        print('Test set after Amputation', X_test.shape, y_test.shape)
        print("X_test Amputed value count of varaible ", X_test.isnull().sum()[variable])
    
    if method == "remove":
        print("Simulate Imputation for NA")
        X_train.dropna(axis=0, inplace=True)
        y_train.dropna(inplace=True)
        print('Train after NA value removal Imputation', X_train.shape, y_train.shape)
        X_test.dropna(axis=0, inplace=True)
        y_test.dropna(inplace=True)
        print('Test after NA value removal Imputation', X_test.shape, y_test.shape)
    if method == "mode":
        print("Simulate Imputation for Mode")
        X_train.fillna(X_train[variable].mode()[0], inplace=True)
        print('Train after Mode Imputation', X_train.shape, y_train.shape)
        X_test.fillna(X_test[variable].mode()[0], inplace=True)
        print('Test after Mode Imputation', X_test.shape, y_test.shape)
    if method == "mean":
        print("Simulate Imputation for Mean")
        X_train.fillna(X_train[variable].mean(), inplace=True)
        print('Train after Mean Imputation', X_train.shape, y_train.shape)
        X_test.fillna(X_test[variable].mean(), inplace=True)
        print('Test after Mean Imputation', X_test.shape, y_test.shape)
    if method == "sample":
        print("Simple Imputation using sampling")
        X_train, X_test = sample_imputation_X(variable, X_train, X_test)
    if method == "bad":
        print("Simulate Imputation for BAD")
        X_train.fillna(fill_dict[variable], inplace=True)
        print('Train  after imputation', X_train.shape, y_train.shape)
        X_test.fillna(fill_dict[variable], inplace=True)
        print('Test  after imputation', X_test.shape, y_test.shape)
        
            
    return X_train , y_train, X_test , y_test

In [14]:
def one_hot_encoding(X_train, X_test):
    categorical_features = X_train.select_dtypes(include=['object']).columns.tolist()
    numerical_cols = X_train.select_dtypes(include=['int', 'float']).columns.tolist()
    ohe = OneHotEncoder(drop='first', sparse=False)
    X_train_encoded = ohe.fit_transform(X_train[categorical_features])
    X_test_encoded = ohe.fit_transform(X_test[categorical_features])
    X_train_new = np.hstack((X_train[numerical_cols].values, X_train_encoded))
    X_test_new = np.hstack((X_test[numerical_cols].values, X_test_encoded))
    return X_train_new, X_test_new

In [15]:
def model_evaluation(X_train,y_train):
    print(datetime.datetime.now())
    model_lr = LinearRegression(fit_intercept=True).fit(X_train, y_train) 
    print(datetime.datetime.now())
    return model_lr

In [16]:
features_for_amputation = ['start_lat','start_lon','end_lat','end_lon','usertype','hour',
                     'temp','feelslike','dew','snowdepth','winddir','sealevelpressure','visibility','solarradiation',
                     'conditions','dist','birthyear','holiday','day','month','gender']

## 1. Drop NA values 

In [15]:
bootstrap_data_drop ={}
for var in features_for_amputation:
    print("Now running model is ....", var)
    X_train, X_test, y_train, y_test = split_dataset(df)
    X_train , y_train, X_test , y_test = ampute_each_variables(var,"remove",X_train,y_train,X_test,y_test)
    X_train, X_test = one_hot_encoding(X_train,X_test)
    X_train = convert_to_dataframe(X_train)
    X_test = convert_to_dataframe(X_test)
    model_lr = LinearRegression(fit_intercept=True)
    rng = np.random.RandomState(1234)
    idx_train = np.arange(y_train.shape[0])
    idx_test = np.arange(y_test.shape[0])
    bootstarp_train_accuracies = {}
    print(f"started bootstrap run is ", datetime.datetime.now())
    for i in range(501):
        train_idx = rng.choice(idx_train, size=idx_train.shape[0], replace=True)
        test_idx = rng.choice(idx_test, size=idx_test.shape[0], replace=True)

        boot_train_X, boot_train_y = X_train.iloc[train_idx], y_train.iloc[train_idx]

        boot_test_X, boot_test_y = X_test.iloc[test_idx], y_test.iloc[test_idx]

        model_lr.fit(boot_test_X, boot_test_y )
        boot_y_pred = model_lr.predict(boot_test_X)
        bootstarp_train_accuracies[i] = {"RMSE": np.sqrt(mean_squared_error(boot_test_y, boot_y_pred)),
                                      "R2":r2_score(boot_test_y, boot_y_pred), "intercept":model_lr.intercept_}
    print(f"stopped bootstrap run is ", datetime.datetime.now())   
    R2 = []
    RMSE = []
    intercept = []
    for i in range(501):
        R2.append(bootstarp_train_accuracies[i]['R2'])
        RMSE.append(bootstarp_train_accuracies[i]['RMSE'])
        intercept.append(bootstarp_train_accuracies[i]['intercept'])
    bootstrap_data_drop[var] = {"R2": [np.percentile(R2, 2.5), np.percentile(R2, 97.5)],
                         "RMSE": [np.percentile(RMSE, 2.5), np.percentile(RMSE, 97.5)],
                         "intercept": [np.percentile(intercept, 2.5), np.percentile(intercept, 97.5)]}

Now running model is .... start_lat
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
sample size is = 25235
Train set after Amputation (252346, 21) (252346,)
y_train Amputed value count of varaible  25235
X_train Amputed value count of varaible  25235
sample size is = 6309
Test set after Amputation (63087, 21) (63087,)
y_test Amputed value count of varaible  6309
X_test Amputed value count of varaible  6309
Simulate Imputation for NA
Train after NA value removal Imputation (227111, 21) (227111,)
Test after NA value removal Imputation (56778, 21) (56778,)
started bootstrap run is  2023-03-19 18:08:53.497023
stopped bootstrap run is  2023-03-19 18:10:13.165630
Now running model is .... start_lon
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
sample size is = 25235
Train set after Amputation (252346, 21) (252346,)
y_train Amputed value count of varaible  25235
X_train Amputed value count of varaible  25235
sample size is = 6309
Test set after Amputation (63087, 21) (63087,)


In [16]:
bootstrap_data_drop

{'start_lat': {'R2': [0.5166782330173632, 0.5332145764136873],
  'RMSE': [2.2138680930642938, 2.2591935575905326],
  'intercept': [-738.2875789927731, -377.6218444229073]},
 'start_lon': {'R2': [0.5166782330173632, 0.5332145764136873],
  'RMSE': [2.2138680930642938, 2.2591935575905326],
  'intercept': [-738.2875789927731, -377.6218444229073]},
 'end_lat': {'R2': [0.5166782330173632, 0.5332145764136873],
  'RMSE': [2.2138680930642938, 2.2591935575905326],
  'intercept': [-738.2875789927731, -377.6218444229073]},
 'end_lon': {'R2': [0.5166782330173632, 0.5332145764136873],
  'RMSE': [2.2138680930642938, 2.2591935575905326],
  'intercept': [-738.2875789927731, -377.6218444229073]},
 'usertype': {'R2': [0.5166782330173632, 0.5332145764136873],
  'RMSE': [2.2138680930642938, 2.2591935575905326],
  'intercept': [-738.2875789927731, -377.6218444229073]},
 'hour': {'R2': [0.5166782330173632, 0.5332145764136873],
  'RMSE': [2.2138680930642938, 2.2591935575905326],
  'intercept': [-738.287578992

### Mode Replacement

In [17]:
bootstrap_data_mode ={}
for var in features_for_amputation:
    print("Now running model is ....", var)
    X_train, X_test, y_train, y_test = split_dataset(df)
    X_train , y_train, X_test , y_test = ampute_each_variables(var,"mode",X_train,y_train,X_test,y_test)
    X_train, X_test = one_hot_encoding(X_train,X_test)
    X_train = convert_to_dataframe(X_train)
    X_test = convert_to_dataframe(X_test)
    model_lr = LinearRegression(fit_intercept=True)
    rng = np.random.RandomState(1234)
    idx_train = np.arange(y_train.shape[0])
    idx_test = np.arange(y_test.shape[0])
    bootstarp_train_accuracies = {}
    print(f"started bootstrap run is ", datetime.datetime.now())
    for i in range(501):
        train_idx = rng.choice(idx_train, size=idx_train.shape[0], replace=True)
        test_idx = rng.choice(idx_test, size=idx_test.shape[0], replace=True)

        boot_train_X, boot_train_y = X_train.iloc[train_idx], y_train.iloc[train_idx]

        boot_test_X, boot_test_y = X_test.iloc[test_idx], y_test.iloc[test_idx]

        model_lr.fit(boot_test_X, boot_test_y )
        boot_y_pred = model_lr.predict(boot_test_X)
        bootstarp_train_accuracies[i] = {"RMSE": np.sqrt(mean_squared_error(boot_test_y, boot_y_pred)),
                                      "R2":r2_score(boot_test_y, boot_y_pred), "intercept":model_lr.intercept_}
    print(f"stopped bootstrap run is ", datetime.datetime.now())  
    
    R2 = []
    RMSE = []
    intercept = []
    for i in range(501):
        R2.append(bootstarp_train_accuracies[i]['R2'])
        RMSE.append(bootstarp_train_accuracies[i]['RMSE'])
        intercept.append(bootstarp_train_accuracies[i]['intercept'])
    bootstrap_data_mode[var] = {"R2": [np.percentile(R2, 2.5), np.percentile(R2, 97.5)],
                         "RMSE": [np.percentile(RMSE, 2.5), np.percentile(RMSE, 97.5)],
                         "intercept": [np.percentile(intercept, 2.5), np.percentile(intercept, 97.5)]}

Now running model is .... start_lat
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
sample size is = 25235
Train set after Amputation (252346, 21) (252346,)
X_train Amputed value count of varaible  25235
sample size is = 6309
Test set after Amputation (63087, 21) (63087,)
X_test Amputed value count of varaible  6309
Simulate Imputation for Mode
Train after Mode Imputation (252346, 21) (252346,)
Test after Mode Imputation (63087, 21) (63087,)
started bootstrap run is  2023-03-19 18:48:46.286719
stopped bootstrap run is  2023-03-19 18:50:02.887673
Now running model is .... start_lon
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
sample size is = 25235
Train set after Amputation (252346, 21) (252346,)
X_train Amputed value count of varaible  25235
sample size is = 6309
Test set after Amputation (63087, 21) (63087,)
X_test Amputed value count of varaible  6309
Simulate Imputation for Mode
Train after Mode Imputation (252346, 21) (252346,)
Test after Mode Imputation (63087, 2

In [18]:
bootstrap_data_mode

{'start_lat': {'R2': [0.5197430133381982, 0.5358373580236255],
  'RMSE': [2.208217091655412, 2.255624032911845],
  'intercept': [-669.6846327727858, -372.4921526976552]},
 'start_lon': {'R2': [0.5194170402686948, 0.5355646529322541],
  'RMSE': [2.208699578399159, 2.25631905096268],
  'intercept': [-642.3337877604681, -342.56840891155355]},
 'end_lat': {'R2': [0.5197848374837518, 0.5358265862237783],
  'RMSE': [2.2081989492521084, 2.255620140442642],
  'intercept': [-681.8147504526933, -386.3841911239268]},
 'end_lon': {'R2': [0.519728911252999, 0.5357929406198051],
  'RMSE': [2.208390263690179, 2.255863544241289],
  'intercept': [-716.4969353576146, -417.39949068221944]},
 'usertype': {'R2': [0.5180404297849346, 0.5339835910463512],
  'RMSE': [2.2125433873858773, 2.2602926382809656],
  'intercept': [-646.2407601386949, -362.1291793349827]},
 'hour': {'R2': [0.5194114772846687, 0.5354203218923754],
  'RMSE': [2.2089528590525194, 2.256421900173301],
  'intercept': [-649.5247626781268, -3

### Mean Replacement

In [19]:
variables_for_mean_amputation = ['start_lat','start_lon','end_lat','end_lon','hour',
                                 'temp','feelslike','snowdepth','winddir','visibility','sealevelpressure',
                                 'solarradiation','dist','birthyear']

bootstrap_data_mean = {}
for var in variables_for_mean_amputation:
    print("Now running model is ....", var)
    X_train, X_test, y_train, y_test = split_dataset(df)
    X_train , y_train, X_test , y_test = ampute_each_variables(var,"mean",X_train,y_train,X_test,y_test)
    X_train, X_test = one_hot_encoding(X_train,X_test)
    X_train = convert_to_dataframe(X_train)
    X_test = convert_to_dataframe(X_test)
    model_lr = LinearRegression(fit_intercept=True)
    rng = np.random.RandomState(1234)
    idx_train = np.arange(y_train.shape[0])
    idx_test = np.arange(y_test.shape[0])
    bootstarp_train_accuracies = {}
    print(f"started bootstrap run is ", datetime.datetime.now())
    for i in range(501):
        train_idx = rng.choice(idx_train, size=idx_train.shape[0], replace=True)
        test_idx = rng.choice(idx_test, size=idx_test.shape[0], replace=True)

        boot_train_X, boot_train_y = X_train.iloc[train_idx], y_train.iloc[train_idx]

        boot_test_X, boot_test_y = X_test.iloc[test_idx], y_test.iloc[test_idx]

        model_lr.fit(boot_test_X, boot_test_y )
        boot_y_pred = model_lr.predict(boot_test_X)
        bootstarp_train_accuracies[i] = {"RMSE": np.sqrt(mean_squared_error(boot_test_y, boot_y_pred)),
                                      "R2":r2_score(boot_test_y, boot_y_pred), "intercept":model_lr.intercept_}
    print(f"stopped bootstrap run is ", datetime.datetime.now())  
    R2 = []
    RMSE = []
    intercept = []
    for i in range(501):
        R2.append(bootstarp_train_accuracies[i]['R2'])
        RMSE.append(bootstarp_train_accuracies[i]['RMSE'])
        intercept.append(bootstarp_train_accuracies[i]['intercept'])
    bootstrap_data_mean[var] = {"R2": [np.percentile(R2, 2.5), np.percentile(R2, 97.5)],
                         "RMSE": [np.percentile(RMSE, 2.5), np.percentile(RMSE, 97.5)],
                         "intercept": [np.percentile(intercept, 2.5), np.percentile(intercept, 97.5)]}

Now running model is .... start_lat
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
sample size is = 25235
Train set after Amputation (252346, 21) (252346,)
X_train Amputed value count of varaible  25235
sample size is = 6309
Test set after Amputation (63087, 21) (63087,)
X_test Amputed value count of varaible  6309
Simulate Imputation for Mean
Train after Mean Imputation (252346, 21) (252346,)
Test after Mean Imputation (63087, 21) (63087,)
started bootstrap run is  2023-03-19 19:28:55.557184
stopped bootstrap run is  2023-03-19 19:30:13.350655
Now running model is .... start_lon
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
sample size is = 25235
Train set after Amputation (252346, 21) (252346,)
X_train Amputed value count of varaible  25235
sample size is = 6309
Test set after Amputation (63087, 21) (63087,)
X_test Amputed value count of varaible  6309
Simulate Imputation for Mean
Train after Mean Imputation (252346, 21) (252346,)
Test after Mean Imputation (63087, 2

In [20]:
bootstrap_data_mean

{'start_lat': {'R2': [0.5197397250785301, 0.5358364156188087],
  'RMSE': [2.2082051579025204, 2.255657351138419],
  'intercept': [-667.6705675737468, -375.7283008547859]},
 'start_lon': {'R2': [0.5194573859131421, 0.5355772659617082],
  'RMSE': [2.2086489727759346, 2.25620859350764],
  'intercept': [-636.4333084855912, -337.74269344246454]},
 'end_lat': {'R2': [0.5197797203071506, 0.5358274900333585],
  'RMSE': [2.208183450530155, 2.255638690579952],
  'intercept': [-682.5664175020434, -387.3886761411452]},
 'end_lon': {'R2': [0.5197405912633163, 0.535794254493799],
  'RMSE': [2.2083679800060265, 2.2558867517579957],
  'intercept': [-717.6976573990769, -419.01315889125084]},
 'hour': {'R2': [0.5196423832083403, 0.5356819742888398],
  'RMSE': [2.2084920987707273, 2.2558250689858204],
  'intercept': [-646.9555555273453, -365.5916952024505]},
 'temp': {'R2': [0.5197554458251847, 0.5357997889263586],
  'RMSE': [2.208420750120516, 2.2557079747220024],
  'intercept': [-645.1333404586464, -36

### Sampling Replacement

In [21]:
bootstrap_data_sample ={}
for var in features_for_amputation:
    print("Now running model is ....", var)
    X_train, X_test, y_train, y_test = split_dataset(df)
    X_train , y_train, X_test , y_test = ampute_each_variables(var,"sample",X_train,y_train,X_test,y_test)
    X_train, X_test = one_hot_encoding(X_train,X_test)
    X_train = convert_to_dataframe(X_train)
    X_test = convert_to_dataframe(X_test)
    model_lr = LinearRegression(fit_intercept=True)
    rng = np.random.RandomState(1234)
    idx_train = np.arange(y_train.shape[0])
    idx_test = np.arange(y_test.shape[0])
    bootstarp_train_accuracies = {}
    print(f"started bootstrap run is ", datetime.datetime.now())
    
    for i in range(501):
        train_idx = rng.choice(idx_train, size=idx_train.shape[0], replace=True)
        test_idx = rng.choice(idx_test, size=idx_test.shape[0], replace=True)

        boot_train_X, boot_train_y = X_train.iloc[train_idx], y_train.iloc[train_idx]

        boot_test_X, boot_test_y = X_test.iloc[test_idx], y_test.iloc[test_idx]

        model_lr.fit(boot_test_X, boot_test_y )
        boot_y_pred = model_lr.predict(boot_test_X)
        bootstarp_train_accuracies[i] = {"RMSE": np.sqrt(mean_squared_error(boot_test_y, boot_y_pred)),
                                      "R2":r2_score(boot_test_y, boot_y_pred), "intercept":model_lr.intercept_}
    print(f"stopped bootstrap run is ", datetime.datetime.now())  
    
    R2 = []
    RMSE = []
    intercept = []
    for i in range(501):
        R2.append(bootstarp_train_accuracies[i]['R2'])
        RMSE.append(bootstarp_train_accuracies[i]['RMSE'])
        intercept.append(bootstarp_train_accuracies[i]['intercept'])
    bootstrap_data_sample[var] = {"R2": [np.percentile(R2, 2.5), np.percentile(R2, 97.5)],
                         "RMSE": [np.percentile(RMSE, 2.5), np.percentile(RMSE, 97.5)],
                         "intercept": [np.percentile(intercept, 2.5), np.percentile(intercept, 97.5)]}

Now running model is .... start_lat
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
sample size is = 25235
Train set after Amputation (252346, 21) (252346,)
X_train Amputed value count of varaible  25235
sample size is = 6309
Test set after Amputation (63087, 21) (63087,)
X_test Amputed value count of varaible  6309
Simple Imputation using sampling
started bootstrap run is  2023-03-19 19:53:08.833176
stopped bootstrap run is  2023-03-19 19:54:26.522948
Now running model is .... start_lon
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
sample size is = 25235
Train set after Amputation (252346, 21) (252346,)
X_train Amputed value count of varaible  25235
sample size is = 6309
Test set after Amputation (63087, 21) (63087,)
X_test Amputed value count of varaible  6309
Simple Imputation using sampling
started bootstrap run is  2023-03-19 19:54:51.841429
stopped bootstrap run is  2023-03-19 19:56:11.122675
Now running model is .... end_lat
Train (252346, 21) (252346,)
Test (630

In [22]:
bootstrap_data_sample

{'start_lat': {'R2': [0.5197312750860954, 0.5358113996615751],
  'RMSE': [2.208283950675378, 2.255631225275442],
  'intercept': [-653.822855815373, -363.7334082043121]},
 'start_lon': {'R2': [0.5192981684340707, 0.535479967425784],
  'RMSE': [2.209118036220254, 2.256632673483737],
  'intercept': [-683.3752944746911, -386.1714592981182]},
 'end_lat': {'R2': [0.5197391297048111, 0.5357788483844792],
  'RMSE': [2.208325622496384, 2.2557100774386942],
  'intercept': [-652.9692490262985, -365.75377551749665]},
 'end_lon': {'R2': [0.5195793632486703, 0.5356806537187389],
  'RMSE': [2.2086519621832923, 2.255931668309404],
  'intercept': [-680.650650320189, -389.3524371277453]},
 'usertype': {'R2': [0.5166635486810084, 0.5325742175408907],
  'RMSE': [2.2161286547903756, 2.2634624267475782],
  'intercept': [-647.5088099851531, -360.88258396473805]},
 'hour': {'R2': [0.5194309813232292, 0.5355232248939532],
  'RMSE': [2.209095461787175, 2.256366445988055],
  'intercept': [-648.2763864732134, -36

## Bad imputation

In [17]:
bootstrap_data_bad ={}
for var in features_for_amputation:
    print("Now running model is ....", var)
    X_train, X_test, y_train, y_test = split_dataset(df)
    X_train , y_train, X_test , y_test = ampute_each_variables(var,"bad",X_train,y_train,X_test,y_test)
    X_train, X_test = one_hot_encoding(X_train,X_test)
    X_train = convert_to_dataframe(X_train)
    X_test = convert_to_dataframe(X_test)
    model_lr = LinearRegression(fit_intercept=True)
    rng = np.random.RandomState(1234)
    idx_train = np.arange(y_train.shape[0])
    idx_test = np.arange(y_test.shape[0])
    bootstarp_train_accuracies = {}
    print(f"started bootstrap run is ", datetime.datetime.now())
    
    for i in range(501):
        train_idx = rng.choice(idx_train, size=idx_train.shape[0], replace=True)
        test_idx = rng.choice(idx_test, size=idx_test.shape[0], replace=True)

        boot_train_X, boot_train_y = X_train.iloc[train_idx], y_train.iloc[train_idx]

        boot_test_X, boot_test_y = X_test.iloc[test_idx], y_test.iloc[test_idx]

        model_lr.fit(boot_test_X, boot_test_y )
        boot_y_pred = model_lr.predict(boot_test_X)
        bootstarp_train_accuracies[i] = {"RMSE": np.sqrt(mean_squared_error(boot_test_y, boot_y_pred)),
                                      "R2":r2_score(boot_test_y, boot_y_pred), "intercept":model_lr.intercept_}
    print(f"stopped bootstrap run is ", datetime.datetime.now())  
    
    R2 = []
    RMSE = []
    intercept = []
    for i in range(501):
        R2.append(bootstarp_train_accuracies[i]['R2'])
        RMSE.append(bootstarp_train_accuracies[i]['RMSE'])
        intercept.append(bootstarp_train_accuracies[i]['intercept'])
    bootstrap_data_bad[var] = {"R2": [np.percentile(R2, 2.5), np.percentile(R2, 97.5)],
                         "RMSE": [np.percentile(RMSE, 2.5), np.percentile(RMSE, 97.5)],
                         "intercept": [np.percentile(intercept, 2.5), np.percentile(intercept, 97.5)]}

Now running model is .... start_lat
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
sample size is = 25235
Train set after Amputation (252346, 21) (252346,)
X_train Amputed value count of varaible  25235
sample size is = 6309
Test set after Amputation (63087, 21) (63087,)
X_test Amputed value count of varaible  6309
Simulate Imputation for BAD
Train  after imputation (252346, 21) (252346,)
Test  after imputation (63087, 21) (63087,)
started bootstrap run is  2023-03-21 14:46:53.651844
stopped bootstrap run is  2023-03-21 14:48:48.740261
Now running model is .... start_lon
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
sample size is = 25235
Train set after Amputation (252346, 21) (252346,)
X_train Amputed value count of varaible  25235
sample size is = 6309
Test set after Amputation (63087, 21) (63087,)
X_test Amputed value count of varaible  6309
Simulate Imputation for BAD
Train  after imputation (252346, 21) (252346,)
Test  after imputation (63087, 21) (63087,)
starte

Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
sample size is = 25235
Train set after Amputation (252346, 21) (252346,)
X_train Amputed value count of varaible  25235
sample size is = 6309
Test set after Amputation (63087, 21) (63087,)
X_test Amputed value count of varaible  6309
Simulate Imputation for BAD
Train  after imputation (252346, 21) (252346,)
Test  after imputation (63087, 21) (63087,)
started bootstrap run is  2023-03-21 15:27:44.459769
stopped bootstrap run is  2023-03-21 15:30:15.043663
Now running model is .... birthyear
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
sample size is = 25235
Train set after Amputation (252346, 21) (252346,)
X_train Amputed value count of varaible  25235
sample size is = 6309
Test set after Amputation (63087, 21) (63087,)
X_test Amputed value count of varaible  6309
Simulate Imputation for BAD
Train  after imputation (252346, 21) (252346,)
Test  after imputation (63087, 21) (63087,)
started bootstrap run is  2023-03-21 15:30

In [18]:
bootstrap_data_bad

{'start_lat': {'R2': [0.5196163873722722, 0.5357481757886795],
  'RMSE': [2.2084624320649704, 2.255894442879072],
  'intercept': [-587.9858966997683, -300.49066041633023]},
 'start_lon': {'R2': [0.5187491318654303, 0.5349590570892713],
  'RMSE': [2.2105147991314467, 2.258236114152259],
  'intercept': [-925.5931508423055, -646.7059671396312]},
 'end_lat': {'R2': [0.5195230025870645, 0.5356150359457579],
  'RMSE': [2.208754600054809, 2.256247929482236],
  'intercept': [-540.1851401039712, -258.75636098952]},
 'end_lon': {'R2': [0.5187721371992587, 0.5349441106689029],
  'RMSE': [2.210247523498259, 2.257859771076351],
  'intercept': [-421.98660848283384, -147.3874070803501]},
 'usertype': {'R2': [0.518121617417438, 0.5340137910765937],
  'RMSE': [2.2124832555570224, 2.260137475794455],
  'intercept': [-646.7511605757373, -362.75533635286865]},
 'hour': {'R2': [0.5188876300495998, 0.5348832076739204],
  'RMSE': [2.210312295811651, 2.2579459571213274],
  'intercept': [-656.4955629405324, -3